# Quandl

```bash
export QUANDL_API_KEY="your key"
```

In [ ]:
from zipline.data.bundles import register, ingest
from zipline.data.bundles.quandl import quandl_bundle

# 注册 Quandl bundle
register(
    'quandl',
    quandl_bundle,
    calendar_name='NYSE',
)

# 执行 ingest
ingest('quandl')

# Yahoo-finance

In [ ]:
"""
Ingesting data from CSV files

https://zipline-trader.readthedocs.io/en/latest/bundles.html#ingesting-data-from-csv-files
"""

import yfinance as yf
import pandas as pd
import tqdm
import os


def download_from_yahoo(
    *symbols: str,
    start_date: str | pd.Timestamp,
    end_date: str | pd.Timestamp,
    data_dir: str,
):
    tframe = 'daily'  # 只支持日线数据
    # 从yfinance下载数据
    data = yf.download(
        symbols,
        start=start_date,
        end=end_date,
        group_by='ticker',
    )

    # 转换为Zipline所需格式
    for symbol in tqdm.tqdm(symbols, desc='Processing'):
        symbol_data: pd.DataFrame = data[symbol].reset_index()
        symbol_data.rename(
            columns={
                'Date': 'date',
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'Close': 'close',
                'Volume': 'volume',
                # 'Adj Close': 'adj_close'
            },
            inplace=True
        )
        parent_dir = os.path.expanduser(f'{data_dir}/{tframe}')
        os.makedirs(parent_dir, exist_ok=True)
        symbol_data.to_csv(f'{parent_dir}/{symbol}.csv', index=False)
    return os.path.expanduser(data_dir)


if __name__ == '__main__':
    from zipline.data.bundles import register, ingest
    from zipline.data.bundles.csvdir import csvdir_equities

    dji = [
        'AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD',
        'IBM',
        'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE',
        'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT', 'XOM'
    ]

    data_dir = '~/.data/yahoo'
    start_date = pd.Timestamp('2000-01-03')
    end_date = pd.Timestamp('2025-03-20')

    csvs_dir = download_from_yahoo(
        *dji, start_date=start_date, end_date=end_date, data_dir=data_dir
    )
    ingest_fn = csvdir_equities(['daily'], csvs_dir)
    register(
        'yahoo',
        ingest_fn,
        calendar_name='NYSE',  # US equities
        start_session=start_date,
        end_session=end_date,
    )

    ingest('yahoo', show_progress=True)
